# Problema de los ascensores

#### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

Primera aproximación al problema, el edificio consta de un único bloque con un único ascensor.

In [6]:
from search import *
import copy

In [7]:
# CONSTANTES
NUM_PLANTAS    = 5
NUM_ASCENSORES = 2
CAP_ASCENSOR   = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [8]:
# CÓDIGOS OPERACIONES
BAJAR=0
SUBIR=1
COGER_PERSONA=2
DEJAR_PERSONA=3

In [64]:
# El estado de un nodo del espacio de exploración se compone de:
# ascensor_pos      -> Planta en la que se encuentra el ascensor
# ascensor_personas -> Personas dentro del ascensor
# personas          -> Lista con planta en la que se encuentra cada persona 
#                      (Pueden tomar el valor de la constante EN_ASCENSOR)
class NodeState:

    def __init__(self, init_personas):
        self.personas = init_personas
        self.ascensor_pos = [0]*NUM_ASCENSORES
        self.ascensor_personas = [[]]*NUM_ASCENSORES
        
    def __eq__(self, nodo):
        return self.personas == nodo.personas and self.ascensor_pos == nodo.ascensor_pos and self.ascensor_personas == nodo.ascensor_personas
   
    def __hash__(self):
        return hash((tuple(self.personas), tuple(self.ascensor_pos), tuple(tuple(x) for x in self.ascensor_personas)))
    
    def __lt__(self,nodo):
        return True

In [56]:
# Definimos la clase para la representación del problema. Las acciones
# contempladas son hacer al ascensor subir o bajar hasta la planta con
# personas (o planta destino de los ocupantes) más cercana y recoger o
# dejar a una persona en una planta.

class AscensoresSimplificado(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal
        self.analizados = 0

    # Codificación acciones:
    # (BAJAR, i, j) -> El ascensor i baja j plantas
    # (SUBIR, i, j) -> El ascensor i sube j plantas
    # (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
    # (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
    def actions(self, state):
        
        accs = list()
        
        for num_ascensor in range(0,NUM_ASCENSORES):
            ascensor_pos      = state.ascensor_pos[num_ascensor]
            ascensor_personas = state.ascensor_personas[num_ascensor]
            personas          = state.personas

            ascensor_vacio    = not ascensor_personas
            ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)

            personas_esta_planta_suben = []
            personas_esta_planta_bajan = []

            for i in range(0,len(personas)):
                if personas[i]==ascensor_pos:
                    if self.goal[i]>ascensor_pos:
                        personas_esta_planta_suben.append(i)
                    if self.goal[i]<ascensor_pos:
                        personas_esta_planta_bajan.append(i)


            # 1.- Si alguien ha llegado a su destino, forzamos que baje del ascensor
            for persona in ascensor_personas:
                if ascensor_pos==self.goal[persona]:
                    return [(DEJAR_PERSONA,num_ascensor,persona)]


            # 2.- Para la acción subir requerimos que haya que coger o dejar a alguien
            #     más arriba de donde se encuentra el ascensor. Nunca subimos con gente
            #     que desea bajar plantas y además, un ascensor no sube si tiene capacidad
            #     para más personas y en esa planta hay alguien que desea subir (eliminación
            #     de ramas sin perder optimaldad).

            persona_menos_arriba = min([person for person in personas if person>ascensor_pos], default=NINGUNA)

            # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
            # y no hay gente en esta planta que quiera subir
            if ascensor_vacio:
                if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                    accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))

            # Si el ascensor no está vacío comprobamos que sus personas suben y vamos
            # al mínimo entre a donde suben estas y donde hay una persona por encima
            # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
            elif self.goal[ascensor_personas[0]]>ascensor_pos:
                destino_menos_arriba = min([self.goal[i] for i in ascensor_personas])

                if ascensor_lleno or not(personas_esta_planta_suben):
                    if persona_menos_arriba==NINGUNA:
                        accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                    else:
                        accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))


            # 3.- La acción bajar es totalmente dual a la acción subir, mismas lógica en
            #     las condiciones. 

            persona_menos_abajo = max([person for person in personas if person<ascensor_pos and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo bajamos si hay alguna persona por abajo
            if ascensor_vacio:
                if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                    accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
            elif self.goal[ascensor_personas[0]]<ascensor_pos:
                destino_menos_abajo = max([self.goal[i] for i in ascensor_personas])

                if ascensor_lleno  or not(personas_esta_planta_bajan):
                    if persona_menos_abajo==NINGUNA:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                    else:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))

            # 4.- Acción dejar persona:
            for personaInterior in ascensor_personas:
                accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))

            # 5.- Acción coger persona. Sólo cogemos a personas que suben o bajan, no ambas
            if not ascensor_lleno:
                if ascensor_vacio:
                    for persona in personas_esta_planta_suben:
                        accs.append((COGER_PERSONA,num_ascensor,persona))
                    for persona in personas_esta_planta_bajan:
                        accs.append((COGER_PERSONA,num_ascensor,persona))
                else:
                    if self.goal[ascensor_personas[0]]<ascensor_pos:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    else:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))

        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        
        ascensor = action[1]
        
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos[ascensor] = estado_nuevo.ascensor_pos[ascensor] - action[2]
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos[ascensor] = estado_nuevo.ascensor_pos[ascensor] + action[2]
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas[ascensor].append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
        else:
            estado_nuevo.ascensor_personas[ascensor].remove(action[2])
            
            if self.goal[action[2]]==state.ascensor_pos[ascensor]:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = state.ascensor_pos[ascensor]
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for persona in state.personas]

    def path_cost(self, c, state1, action, state2):
        if action[0]==BAJAR or action[0]==SUBIR:
            return c + action[2]
        else:
            return c
    
    def h(self,node):
        suma  = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        #suma += sum([abs(self.goal[i]-node.state.ascensor_personas[i]) for i in range(0,len(node.state.ascensor_personas))])
        return suma
        
    def value(self, state):
        raise NotImplementedError

In [57]:
# Codificación acciones:
# (BAJAR, i, j) -> El ascensor i baja j plantas
# (SUBIR, i, j) -> El ascensor i sube j plantas
# (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
# (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
def decodificador_acciones(acciones):
    for accion in acciones:
        if accion[0]==BAJAR:
            print("El ascensor "+str(accion[1])+" baja "+str(accion[2])+" plantas")
        elif accion[0]==SUBIR:
            print("El ascensor "+str(accion[1])+" sube "+str(accion[2])+" plantas")
        elif accion[0]==COGER_PERSONA:
            print("El ascensor "+str(accion[1])+" coge a la persona "+str(accion[2]))
        else:
            print("El ascensor "+str(accion[1])+" deja a la persona "+str(accion[2]))

In [58]:
def fusionador_acciones(acciones):
    nuevas_acciones = []
    for accion in acciones:
        if bool(nuevas_acciones) and (accion[0]==nuevas_acciones[-1][0] and accion[1]==nuevas_acciones[-1][1]) and (accion[0]==BAJAR or accion[0]==SUBIR):
            nuevas_acciones[-1] = (nuevas_acciones[-1][0],nuevas_acciones[-1][1],nuevas_acciones[-1][2]+accion[2])
        else:
            nuevas_acciones.append(accion)
    return nuevas_acciones

In [59]:
def coste_acciones(acciones):
    coste = 0
    for action in acciones:
        if action[0]==BAJAR or action[0]==SUBIR:
            coste += action[2]
    return coste

In [60]:
#def AscensoresSimplificadoConAnalizados(Problem):
#    
#    def __init__(self, problem, goal=None):
#        self.problem = problem
#        self.initial = problem.initial
#        self.analizados = 0
#        self.goal = problem.goal
#        
#    def actions(self, estado):
#        return self.problem.actions(estado)
#
#    def result(self, estado, accion):
#        return self.problem.result(estado, accion)
#
#    def goal_test(self, estado):
#        self.analizados += 1
#        return self.problem.goal_test(estado)

In [61]:
init = NodeState([0,0,0])
goal = [1,1,1]

In [62]:
problem = AscensoresSimplificado(init, goal)

In [63]:
decodificador_acciones(fusionador_acciones(breadth_first_graph_search(problem).solution()))

El ascensor 0 coge a la persona 0
El ascensor 0 coge a la persona 1
El ascensor 0 sube 1 plantas
El ascensor 0 deja a la persona 0
El ascensor 0 deja a la persona 1
El ascensor 1 coge a la persona 2
El ascensor 0 deja a la persona 2


In [47]:
problem.analizados

10

In [30]:
problem.analizados=0

In [31]:
acciones = fusionador_acciones(astar_search(problem).solution())
decodificador_acciones(acciones)

El ascensor 0 sube 1 plantas
El ascensor 0 coge a la persona 4
El ascensor 0 coge a la persona 7
El ascensor 0 sube 3 plantas
El ascensor 0 deja a la persona 7
El ascensor 0 coge a la persona 1
El ascensor 0 sube 4 plantas
El ascensor 0 deja a la persona 4
El ascensor 0 coge a la persona 4
El ascensor 0 sube 1 plantas
El ascensor 0 deja a la persona 4
El ascensor 0 deja a la persona 1
El ascensor 0 baja 1 plantas
El ascensor 0 coge a la persona 3
El ascensor 0 baja 1 plantas
El ascensor 0 coge a la persona 6
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 6
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 3
El ascensor 0 coge a la persona 2
El ascensor 0 sube 1 plantas
El ascensor 0 coge a la persona 0
El ascensor 0 sube 1 plantas
El ascensor 0 deja a la persona 0
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 2
El ascensor 0 coge a la persona 5
El ascensor 0 baja 1 plantas
El ascensor 0 deja a la persona 5
El ascensor 0 sube 1 plantas
El ascensor

In [32]:
problem.analizados

8405

In [33]:
coste_acciones(acciones)

86

In [10]:
[[1,2,3],[1,2]]==[[1,2,3],[1,1]]

False